In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

In [ ]:
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Prediction-key': '009281becef44f2c85dd96cd749d56ea',
}

In [ ]:
params = urllib.parse.urlencode({
    # Request parameters
    'iterationId': '22f47df0-85a5-42e2-9e0c-d73a1591ad1d',
    'application': 'task1',
})

In [ ]:
try:
    conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
    conn.request("POST", "/customvision/v2.0/Prediction/dbb69137-acbf-4209-88dd-3220c4011eb3/url?%s" % params, '{"Url": "http://img.cdn.mountainwarehouse.com/product/025374/025374_pur_featherweight_down_womens_jacket__wms_aw17_2.jpg"}', headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))


In [ ]:
import os
from os import listdir, makedirs
from os.path import isfile, join
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
from PIL import ImageOps
desired_size = 128
from matplotlib.pyplot import imshow
new_img = Image.new("RGB",(1,1),(0,0,0))

def process_image(old_im_pth,new_im_path):
    im = Image.open(old_im_pth)
    old_size = im.size  

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
   
    im = im.resize(new_size, Image.ANTIALIAS)
   
    new_im = Image.new("RGB", (desired_size, desired_size), "white")
    new_im.paste(im, ((desired_size-new_size[0])//2,
                     (desired_size-new_size[1])//2))
    new_im.save(new_im_path, "JPEG")
    
def normalize(arr):
    """
    Linear normalization
    http://en.wikipedia.org/wiki/Normalization_%28image_processing%29
    """
    arr = arr.astype('float')
    # Do not touch the alpha channel
    for i in range(3):
        minval = arr[...,i].min()
        maxval = arr[...,i].max()
        if minval != maxval:
            arr[...,i] -= minval
            arr[...,i] *= (255.0/(maxval-minval))
    return arr

def save_equalize(image_path, new_image_path):
    img = Image.open(image_path).convert('RGB')
    img.show
    plt.imshow(img)
    plt.title('Before Equalize')
    plt.show()
    
    #histogram
    mp_img=mpimg.imread(image_path)
    plt.style.use('seaborn-white')
    plt.hist(mp_img.ravel(), 256, [0, 256])
    plt.show()
    
    new_img = ImageOps.equalize(img)
    plt.imshow(new_img)
    plt.title('After Equalize')
    plt.show()

    #histogram
    mp_img=mpimg.imread(new_image_path)
    plt.style.use('seaborn-white')
    plt.hist(mp_img.ravel(), 256, [0, 256])
    plt.show()

def save_normalize(image_path, new_image_path):
    global new_img
    img = Image.open(image_path).convert('RGB')
    arr = np.array(img)
    imshow(arr)
    new_img = Image.fromarray(normalize(arr).astype('uint8'),'RGB')
    new_img.save(new_image_path)

img_root = '/home/team12/team12/data/gear_images'
img_dirs = [os.path.join(img_root, o) for o in os.listdir(img_root) if os.path.isdir(os.path.join(img_root,o))]
for img_dir in img_dirs:
    img_files = [f for f in listdir(img_dir) if isfile(join(img_dir, f))]
    for img_file in img_files:
    img_dir = img_root + "/boots"
    img_file = "733952.jpeg"
    print(img_dir + '/' + img_file)
    image_path = img_dir + '/' + img_file

    #step 1 process image with pad and resize - completed
    save_dir = img_dir + '/step1/'
    new_image_path_step1 = save_dir + img_file

    #step 2a is normalize
    save_dir = img_dir + '/step3/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    new_image_path_step = save_dir + img_file
    #process_image(image_path,new_image_path_step)
    save_normalize(new_image_path_step1, new_image_path_step)
    print('Step 3 normalized and saved to: ' + new_image_path_step)

In [ ]:
def to2D(arr):
    x, y, z = arr.shape
    indices = np.vstack(np.unravel_index(np.arange(x*y), (y, x))).T
    return np.hstack((arr.reshape(x*y, z), indices))

In [2]:
from PIL import Image, ImageOps
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split

img_root = '/home/team12/team12/data/gear_images_normalized'
img_dirs = [os.path.join(img_root, o) for o in os.listdir(img_root) if os.path.isdir(os.path.join(img_root,o))]
img_dirs.sort()

x = []
y = []
category = 0
for img_dir in img_dirs:
    print(img_dir + " : " + str(category))
    count = 0
    img_files = [f for f in listdir(img_dir) if isfile(join(img_dir, f))]
    for img_file in img_files:
        #print(img_dir + '/' + img_file)
        image_path = img_dir + '/' + img_file
        x.append(np.array(Image.open(image_path).convert("RGB")))
        y.append(category)
        #if count > 100:
        #    break
        count += 1
    category += 1

X = np.array(x) #.reshape(len(x), -1)
Y = np.array(y)

/home/team12/team12/data/gear_images_normalized/axes : 0
/home/team12/team12/data/gear_images_normalized/boots : 1
/home/team12/team12/data/gear_images_normalized/carabiners : 2
/home/team12/team12/data/gear_images_normalized/crampons : 3
/home/team12/team12/data/gear_images_normalized/gloves : 4
/home/team12/team12/data/gear_images_normalized/hardshell_jackets : 5
/home/team12/team12/data/gear_images_normalized/harnesses : 6
/home/team12/team12/data/gear_images_normalized/helmets : 7
/home/team12/team12/data/gear_images_normalized/insulated_jackets : 8
/home/team12/team12/data/gear_images_normalized/pulleys : 9
/home/team12/team12/data/gear_images_normalized/rope : 10
/home/team12/team12/data/gear_images_normalized/tents : 11


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
X_train = (X_train/255).astype('float32')
X_test = (X_test/255).astype('float32')

In [ ]:
#import matplotlib.pyplot as plt
#predictions = clf.predict(X_test)
#plt.scatter(y_test, predictions)
#plt.xlabel('True Values')
#plt.ylabel('Predictions')
#plt.show()
#print("Score:", clf.score(X_test, y_test))

In [ ]:
#from sklearn import tree
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import ExtraTreesClassifier

#clf = tree.DecisionTreeClassifier(max_depth=11)
#clf = RandomForestClassifier()
#clf = ExtraTreesClassifier(n_estimators=22,max_depth=11,random_state=40)

#clf = clf.fit(X_train, y_train)
#predictions = clf.predict(X_test)
#print("Score:", clf.score(X_test, y_test))

In [ ]:
#import graphviz 
#dot_data = tree.export_graphviz(clf, out_file=None) 
#graph = graphviz.Source(dot_data) 
#graph.render("gears") 

In [5]:
import keras
from keras.models import Sequential
model = Sequential()
from keras.layers import Dense,Activation,Flatten,Conv2D, MaxPooling2D,Dropout,Reshape, BatchNormalization

model.add(Conv2D(12, (8,8), activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (16,16), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.01))

model.add(Flatten())
model.add(Dense(12, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
        optimizer='sgd',metrics=['accuracy'])

from keras.utils.np_utils import to_categorical

import tensorflow as tf
#with tf.device('/gpu:0'):
model.fit(X_train, to_categorical(y_train), epochs=20)

#model.summary()
model.evaluate(X_test, to_categorical(y_test), batch_size=30)

Epoch 1/20
1697/1697 [==============================] - 2s 1ms/step - loss: 2.9032 - acc: 0.6317
Epoch 2/20
1697/1697 [==============================] - 2s 904us/step - loss: 1.5477 - acc: 0.7926
Epoch 3/20
1697/1697 [==============================] - 2s 902us/step - loss: 0.8321 - acc: 0.8668
Epoch 4/20
1697/1697 [==============================] - 2s 933us/step - loss: 0.8300 - acc: 0.8674
Epoch 5/20
1697/1697 [==============================] - 2s 943us/step - loss: 0.6319 - acc: 0.8922
Epoch 6/20
1697/1697 [==============================] - 2s 926us/step - loss: 0.4086 - acc: 0.9228
Epoch 7/20
1697/1697 [==============================] - 2s 934us/step - loss: 0.4700 - acc: 0.9134
Epoch 8/20
1697/1697 [==============================] - 2s 907us/step - loss: 0.1844 - acc: 0.9652
Epoch 9/20
1697/1697 [==============================] - 2s 906us/step - loss: 1.2877 - acc: 0.8138
Epoch 10/20
1697/1697 [==============================] - 2s 929us/step - loss: 0.5640 - acc: 0.8922
Epoch 11/20

[0.5806271637187285, 0.8682352809345021]

In [1]:
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from keras import backend as K

num_cores = 1

import tensorflow as tf
config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
    inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
    device_count = {'CPU' : 1, 'GPU' : 2})

session = tf.Session(config=config)
K.set_session(session)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12480422603762375889
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11287530701
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8230583413719199335
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 1abd:00:00.0, compute capability: 3.7"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 11287530701
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18262272416983403501
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 7ff0:00:00.0, compute capability: 3.7"
]


In [ ]:
from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.models import load_model
from PIL import Image
import numpy as np
import flask
import io

# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None
lookup = {0: 'axes', 1: 'boots', 2: 'carabiners', 3: 'crampons', 4: 'gloves', 5: 'hardshell_jackets',
          6: 'harnesses', 7: 'helmets', 8: 'insulated_jackets', 9: 'pulleys', 10: 'rope', 11: 'tents'}


def load_model_s():
    # load the pre-trained Keras model (here we are using a model
    # pre-trained on ImageNet and provided by Keras, but you can
    # substitute in your own networks just as easily)
    global model
    #model = ResNet50(weights="imagenet")
    model = load_model('cnn-aw-model.h5')


def prepare_image(image, target):
    # if the image mode is not RGB, convert it
    if image.mode != "RGB":
        image = image.convert("RGB")

    # resize the input image and preprocess it
    image = image.resize(target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)

    # return the processed image
    return image


@app.route("/predict", methods=["POST"])
def predict():
    # initialize the data dictionary that will be returned from the
    # view
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    if flask.request.method == "POST":
        if flask.request.files.get("image"):
            # read the image in PIL format
            image = flask.request.files["image"].read()
            image = Image.open(io.BytesIO(image))

            # preprocess the image and prepare it for classification
            image = prepare_image(image, target=(128, 128))

            # classify the input image and then initialize the list
            # of predictions to return to the client
            preds = model.predict(image)[0]
            #results = imagenet_utils.decode_predictions(preds)
            #preds = preds.astype('uint8')
            print(preds)
            data["predictions"] = np.array_str(preds)

            # loop over the results and add them to the list of
            # returned predictions
            # for (imagenetID, label, prob) in preds[0]:
            #           r = {"label": label, "probability": float(prob)}
     #          data["predictions"].append(r)

            # indicate that the request was a success
            data["success"] = True

    # return the data dictionary as a JSON response
    return flask.jsonify(data)


# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
           "please wait until server has fully started"))
    load_model_s()
    app.run(host='0.0.0.0:5002')